In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Set the working directory
os.chdir("C:\TradeGroup")
my_wd = os.getcwd()
new_directory = my_wd

def create_folder(folder_path):
    if not os.path.exists(f'{new_directory}/{folder_path}'):
        os.makedirs(f'{new_directory}/{folder_path}')
    else:
        print(f'Already Exists')

create_folder('Data/Indices')

Already Exists


In [3]:
Iran_Exports = pd.read_excel(f'{my_wd}/Data/Indices/Trademap_IranExports_2022.xlsx')
Iran_Exports['Code'] = Iran_Exports['Code'].str.replace("'","").astype('int')
#Iran_Exports['Code'] = Iran_Exports['Code'].astype("int")

remaining_columns = Iran_Exports.columns[2:]


remaining_columns=remaining_columns.str.replace("Exported value in ","")

Iran_Exports.columns=Iran_Exports.columns.str.replace("Exported value in ","")


Iran_Exports = Iran_Exports[Iran_Exports['Code']!=999999]

Iran_Exports.columns = Iran_Exports.columns[0:2].tolist() + [col + '_Iran_Exports' for col in Iran_Exports.columns[2:]]

In [4]:
new_columns = [f'Share_{year}' for year in Iran_Exports.columns[2:]]
Share_Iran = pd.DataFrame(columns=['Code'] + new_columns)
Share_Iran['Code']=Iran_Exports['Code']

for year in range(2003,2023):
    Share_Iran[f'Share_{year}_Iran_Exports']=Iran_Exports[f'{year}_Iran_Exports']/(Iran_Exports[f'{year}_Iran_Exports'].sum())   

 

In [6]:
Countries = ['China','Turkey','Russia','SKorea','Iraq', 'Indonasia','Syria','Armenia' ,'Belarus', 'Kazakhstan', 'Kyrgyzstan']
Iran_TCI=pd.DataFrame({"Country": Countries})

for country in Countries:

    Country_Imports=f'{country}_Imports'
    Share_Country=f'Share_{country}'
    Share_Country_Iran_Merged=f'Share_{country}_Iran_Merged'
    Share_diffs= f'{country}_Share_diff'
    print(Country_Imports)

    globals()[Country_Imports]=pd.read_excel(f'{my_wd}/Data/Indices/Trademap_{country}Imports_2023.xlsx')
    
    if (globals()[Country_Imports]['Code'].dtype != "int") and (globals()[Country_Imports]['Code'].dtype == "object"):
        globals()[Country_Imports]['Code'] = globals()[Country_Imports]['Code'].str.replace("'","").astype('int')
    #globals()[Country_Imports]['Code'] = globals()[Country_Imports]['Code'].astype("int")
    
    remaining_columns = globals()[Country_Imports].columns[1:]
    remaining_columns=remaining_columns.str.replace("Imported value in ","")

    globals()[Country_Imports].columns=globals()[Country_Imports].columns.str.replace("Imported value in ","")


    globals()[Country_Imports] = globals()[Country_Imports][globals()[Country_Imports]['Code']!=999999]

    globals()[Country_Imports].columns = globals()[Country_Imports].columns[0:1].tolist() + [col + f'_{country}_Imports' for col in globals()[Country_Imports].columns[1:]]

    new_columns = [f'Share_{year}' for year in globals()[Country_Imports].columns[1:]]
    globals()[Share_Country] = pd.DataFrame(columns=['Code'] + new_columns)
    globals()[Share_Country]['Code']=globals()[Country_Imports]['Code']

    globals()[Share_diffs]=pd.DataFrame()

    if (country!='Iraq') & (country!='Armenia') & (country!='Belarus') & (country!='Kazakhstan') & (country!='Kyrgyzstan'):

        for year in range(2004,2023):
            globals()[Share_Country][f'Share_{year}_{country}_Imports']=globals()[Country_Imports][f'{year}_{country}_Imports']/(globals()[Country_Imports][f'{year}_{country}_Imports'].sum()) 
        
        globals()[Share_Country_Iran_Merged]=Share_Iran.merge(globals()[Share_Country],on=['Code'],how='outer')
        #.fillna(0)

        globals()[Share_diffs]['Code']= globals()[Share_Country_Iran_Merged]['Code']

        for year in range (2009,2023):
            
            globals()[Share_diffs][f'diff_{year}']=abs(globals()[Share_Country_Iran_Merged][f'Share_{year}_{country}_Imports']-globals()[Share_Country_Iran_Merged][f'Share_{year}_Iran_Exports'])/2

            TCI_Year=100*(1-((globals()[Share_diffs][f'diff_{year}'].sum())))

            Iran_TCI.loc[Iran_TCI['Country']==country,f'{year}']=TCI_Year
        


    else:
        for year in range(2014,2023):
            globals()[Share_Country][f'Share_{year}_{country}_Imports']=globals()[Country_Imports][f'{year}_{country}_Imports']/(globals()[Country_Imports][f'{year}_{country}_Imports'].sum()) 
        
        globals()[Share_Country_Iran_Merged]=Share_Iran.merge(globals()[Share_Country],on=['Code'],how='outer')
        #.fillna(0)

        globals()[Share_diffs]['Code']= globals()[Share_Country_Iran_Merged]['Code']

        for year in range (2014,2023):
            
            globals()[Share_diffs][f'diff_{year}']=abs(globals()[Share_Country_Iran_Merged][f'Share_{year}_{country}_Imports']-globals()[Share_Country_Iran_Merged][f'Share_{year}_Iran_Exports'])/2
            
            TCI_Year=100*(1-((globals()[Share_diffs][f'diff_{year}'].sum())))

            Iran_TCI.loc[Iran_TCI['Country']==country,f'{year}']=TCI_Year
    



China_Imports
Turkey_Imports
Russia_Imports
SKorea_Imports
Iraq_Imports
Indonasia_Imports
Syria_Imports
Armenia_Imports
Belarus_Imports
Kazakhstan_Imports
Kyrgyzstan_Imports


***

The numbers are very different

***


In [7]:
Iran_TCI.to_excel(f'{my_wd}\Data\Indices\Iran_TCI.xlsx',index=False)
Iran_TCI

,Country,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,18.802376,18.691007,19.932351,21.154790,22.928365,23.981728,20.659349,20.261407,22.776999,24.548525,24.758370,20.840661,23.947277,27.928839
1,Turkey,11.321858,10.121215,10.592629,9.598925,10.973198,12.083719,14.677095,14.183252,14.334920,14.930147,14.269362,14.352021,20.751919,19.345239
2,Russia,9.801717,8.082633,7.660815,9.021129,10.433835,11.832314,11.626680,9.868075,10.074373,10.638422,12.755158,12.410935,10.694171,20.296641
3,SKorea,26.243185,26.590889,28.865183,30.617602,30.661313,29.637680,25.174185,27.502574,29.786477,30.960991,26.200698,21.535951,30.448015,31.368108
4,Iraq,NaN,NaN,NaN,NaN,NaN,12.638767,43.398750,25.115253,19.675326,23.916521,25.611773,39.291644,29.175786,24.477159
5,Indonasia,27.069842,16.745606,15.854234,16.721103,18.982129,20.084823,20.796112,21.451407,19.682889,20.087424,15.770288,15.192751,21.159915,22.123119
6,Syria,11.264842,9.477104,16.765939,49.275629,47.263793,46.192167,42.527952,23.346796,45.050118,46.670302,26.516871,39.477899,46.548929,42.216510
7,Armenia,NaN,NaN,NaN,NaN,NaN,13.241092,15.155278,15.593838,14.257077,15.643989,14.570316,15.018817,18.144271,19.447849
8,Belarus,NaN,NaN,NaN,NaN,NaN,30.598831,31.934344,26.603851,27.959035,31.338573,30.524287,25.647647,12.260969,24.089342
9,Kazakhstan,NaN,NaN,NaN,NaN,NaN,12.680219,14.697199,14.150746,14.075464,13.807739,14.001856,14.180314,14.259098,15.276165


In [7]:
Iran_TCI

,Country,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,18.802376,18.691007,19.932351,21.154790,22.928365,23.981728,20.659349,20.261407,22.776999,24.548525,24.758370,20.840661,23.947277,27.928839
1,Turkey,11.321858,10.121215,10.592629,9.598925,10.973198,12.083719,14.677095,14.183252,14.334920,14.930147,14.269362,14.352021,20.751919,19.345239
2,Russia,9.801717,8.082633,7.660815,9.021129,10.433835,11.832314,11.626680,9.868075,10.074373,10.638422,12.755158,12.410935,10.694171,20.296641
3,SKorea,26.243185,26.590889,28.865183,30.617602,30.661313,29.637680,25.174185,27.502574,29.786477,30.960991,26.200698,21.535951,30.448015,31.368108
4,Iraq,NaN,NaN,NaN,NaN,NaN,12.638767,43.398750,25.115253,19.675326,23.916521,25.611773,39.291644,29.175786,24.477159
5,Indonasia,27.069842,16.745606,15.854234,16.721103,18.982129,20.084823,20.796112,21.451407,19.682889,20.087424,15.770288,15.192751,21.159915,22.123119
6,Syria,11.264842,9.477104,16.765939,49.275629,47.263793,46.192167,42.527952,23.346796,45.050118,46.670302,26.516871,39.477899,46.548929,42.216510


In [8]:
Iran_TCI.to_excel(f'{my_wd}\Data\Indices\Iran_TCI.xlsx',index=False)
Iran_TCI


,Country,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,18.802376,18.691007,19.932351,21.154790,22.928365,23.981728,20.659349,20.261407,22.776999,24.548525,24.758370,20.840661,23.947277,27.928839
1,Turkey,11.321858,10.121215,10.592629,9.598925,10.973198,12.083719,14.677095,14.183252,14.334920,14.930147,14.269362,14.352021,20.751919,19.345239
2,Russia,9.801717,8.082633,7.660815,9.021129,10.433835,11.832314,11.626680,9.868075,10.074373,10.638422,12.755158,12.410935,10.694171,20.296641
3,SKorea,26.243185,26.590889,28.865183,30.617602,30.661313,29.637680,25.174185,27.502574,29.786477,30.960991,26.200698,21.535951,30.448015,31.368108
4,Iraq,NaN,NaN,NaN,NaN,NaN,12.638767,43.398750,25.115253,19.675326,23.916521,25.611773,39.291644,29.175786,24.477159
5,Indonasia,27.069842,16.745606,15.854234,16.721103,18.982129,20.084823,20.796112,21.451407,19.682889,20.087424,15.770288,15.192751,21.159915,22.123119
6,Syria,11.264842,9.477104,16.765939,49.275629,47.263793,46.192167,42.527952,23.346796,45.050118,46.670302,26.516871,39.477899,46.548929,42.216510
